# Single iteration example
Once you're happy with the first version of your functionality, you can build a scheduler around this to run on a regular basis or listen for events. The simplest possible version of a scheduler is a while loop. 

## Setup openai client
We use Openai's GPT-4o model for this example, but you are free to use any model you want.

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [6]:

from openai import OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# if you're getting an error here, ensure your envirornment is setup and that you have exported an openai api key
# export OPENAI_API_KEY="your_api_key_here"

## Setup hyperliquid sdk 
We use the hyperliquid L1 netowrk for this example, but you are free to use any app/chain you want.
Hyperliquid offers a great SDK for python, fast settlement times for trades, and friendly user interface.

In [7]:
from hyperliquid.info import Info
from hyperliquid.utils import constants
import json

info = Info(constants.TESTNET_API_URL, skip_ws=True)

# This is an example address, when you start trading this should be replaced with your address
user_state = info.user_state("0xcd5051944f780a621ee62e39e493c489668acf4d") 
print(json.dumps(user_state, indent=2))

# if you're getting an error or no user_state here make sure your environment variables are set correctly and that all requirements are installed

{
  "marginSummary": {
    "accountValue": "326762.754905",
    "totalNtlPos": "53182.20944",
    "totalRawUsd": "350864.329725",
    "totalMarginUsed": "2684.400327"
  },
  "crossMarginSummary": {
    "accountValue": "326762.754905",
    "totalNtlPos": "53182.20944",
    "totalRawUsd": "350864.329725",
    "totalMarginUsed": "2684.400327"
  },
  "crossMaintenanceMarginUsed": "531.822092",
  "withdrawable": "324078.354578",
  "assetPositions": [
    {
      "type": "oneWay",
      "position": {
        "coin": "ATOM",
        "szi": "491.45",
        "leverage": {
          "type": "cross",
          "value": 20
        },
        "entryPx": "10.9725",
        "positionValue": "5013.7729",
        "unrealizedPnl": "-378.662225",
        "returnOnEquity": "-1.40442014",
        "liquidationPx": null,
        "marginUsed": "250.688645",
        "maxLeverage": 50,
        "cumFunding": {
          "allTime": "-1698.946681",
          "sinceOpen": "-1698.946681",
          "sinceChange": "

## Define example identity of a user
This can be personalized for each individual user. We'll end up passing these fields to our prompts later, so that the same agent system can be used by many people with different goals and values. 


To think about: what other fields might be useful here? Are there sources of data you could gather from users instead of asking these questions directly? Ex: If I gave you my wallet address, what info would be helpful there? How would you gather and analyze that?

In [3]:

identity_map = {
    "risk_level": "Moderate risk, willing to risk some money for the right investments but not chasing every new opportunity.",
    "token_preferences": "Likes ETH more than BTC, doesnt like SOL",
    "mission_statement": "Accumulate as much ETH as possible given the available funds.",
}

## Gather onchain data for the user's wallet
Info on prices, balances, available assets, etc.


In [4]:
# the amount available to withdraw is approximately equal to the balance available to trade
# there is room for a better way to track this
available_balance = user_state.get('withdrawable') # denominated in USD

In [5]:
# We're also going to take a look at the current positions that are open, and format them for use with our models
positions = user_state.get('assetPositions')
positions_for_llm = ''
if positions:
    for position in positions:
        position = position.get('position')
        position_for_llm = f"Current {position.get('coin')} position: size {position.get('szi')} {position.get('coin')}, value of {position.get('positionValue')} usd, leverage {position.get('leverage').get('value')}, and unrealizedPnl {position.get('unrealizedPnl')} usd. The max leverage for this position is {position.get('maxLeverage')}.\n"
        positions_for_llm += position_for_llm

## Gather all additional information
This is a great place to experiment with different data sources, for this example we'll simulate a summarization of a chat users might have with in chat, and some recent headlines.

In [6]:
recent_headlines = ("*ROBINHOOD: CRYPTO TRADING VOLUMES OVER $30B, UP 600% YOY\n"
                     "*FED’S MUSALEM: TIME MAY BE APPROACHING TO SLOW OR PAUSE RATE CUTS\n")

chat_summary = "Bob is happy with the Ethereum roadmap and has been hearing more people talk about it.\n"

chat_and_data_summary = chat_summary + recent_headlines

## Create our prompts

We do this with markdown because OpenAI models respond well to this format, however you can use any format you like.

This is structured as a system prompt that remains fixed for all users, and a user prompt unique to each user.

In [7]:
system_prompt = (
    "# Instructions:\n"
    "Act as a helpful cryptocurrency assistant helping users manage their trading portfolio.\n"
    "They understand it is inherently risky to trade cryptocurrency, and they want to make sure they are making informed decisions.\n"
    "You will be given a `available_balance` representing the user's total available value to make new trades with, a list of available assets, a risk level, and a list of their current positions.\n"
    "Think carefully through all scenarios and please provide your best guidance and reasoning for this decision.\n"
    "The USD value of each individual trade should not exceed the `available_balance`, and trades should be sized to allow for sufficient 'available_balance' to handle market volatility or unforeseen events.\n"
    "Do not suggest or provide reasoning for order where your suggested order size (for both new and addition to existing positions) is less than 10 USD.\n"
    "Ensure that there is enough margin available to support the trade size and leverage. Adjust leverage or order size accordingly, if required, while remaining within the 10 USD per order limit. If not possible, then do not suggest a new position and instead recommend to the user to deposit additional funds.\n"
    "# Available Options:\n"
    "- create a new position which should be tracked in the list ```positions_to_open```\n"
    "- modify or close an existing position which should be tracked in the list ```positions_to_modify```\n"
    "- maintain an existing position without changes which should be tracked in the list ```positions_to_maintain```\n"
    "# Fields for each option:\n"
    "- asset: the asset to trade\n"
    "\t- example: ETH\n"
    "- direction: the direction to trade\n"
    "\t- example: long, short\n"
    "- size: the size of the trade denominated in USD. It has to be bigger than 10 and should not use up the entire 'available_balance', leaving enough funds available for risk management and flexibility.\n"
    "\t- the trade size should be greater than 10 USD even when modifying an existing position.\n"
    "\t- example: 90 # If the 'available_balance' is 90, use at most 80 for the sum of all trades, keeping 10 as a buffer. Ensure trades are sized to allow for sufficient 'available_balance' to handle market volatility or unforeseen events.\n"
    "- leverage: the leverage to use for the trade\n"
    "\t- example: 10\n"
    "- reasoning: the reasoning for the decision\n"
    "\t- example: ['People value Alice's opinion and she really likes ETH here.', 'ETH price is low right now, volume is high compared to yesterday.', 'ETH is a solid long term investment.']\n"
)

In [8]:
user_message = (
    "# Instructions:\n"
    "Here are some details about me, can you help me make decisions about my trading portfolio?\n"
    "# Personality\n"
    f"{identity_map.get('chat_personality')}\n"
    "# Risk Level\n"
    f"{identity_map.get('risk_level')}\n"
    "This represents the total $USD value of the account, including positions, margin, and available funds.\n"
    "# Available Balance\n"
    f"{available_balance}\n"
    "Portions of this 'available_balance' can be used for placing new orders or modifying existing positions.\n"
    "Always leave a fraction of total 'available_balance' as a safety buffer for unforeseen volatility.\n"
    "The 'available_balance' is shared by all positions, so it is important to keep track of the available value and adjust your position sizes accordingly.\n"
    "# Open Positions\n"
    f"{positions_for_llm}\n"
    "# Here is the most recent information I want to base my decisions on:\n"
    f"{chat_and_data_summary}\n"
)



In [9]:
# append our messages to the chat
model = "gpt-4o-2024-08-06"
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_message},
]


# Define our response format

We don't want to receive a response of unstructured text, so we'll define a custom response format.
We need to be able to act on this information, and need the result in a consistent format. 

You could choose to use tool calling here as well, however we choose not to because we don't always need to call a tool. Ex: if we're maintaining a positioin we might as well do nothing, instead of calling some position to maintain tool.


You can read more about structured output vs. tool calling [here](https://platform.openai.com/docs/guides/structured-outputs)

In [10]:
from pydantic import BaseModel, Field
from typing import Literal

# This class will be used to submit the trades
class Position(BaseModel):
    market: str = Field(..., description="The asset to trade")
    direction: Literal["long", "short"] = Field(
        ...,
        description="The direction to trade",
    )
    size: float = Field(
        ...,
        description="The size of the trade denominated in USD. It should be greater than 10.",
    )
    reasoning: list[str] = Field(
        default_factory=list,
        description="The reasoning for the decision",
    )
    leverage: int | None = Field(None, description="Optional leverage multiplier")


class PositionReasoning(BaseModel):
    positions_to_maintain: list[Position] = Field(
        default_factory=list,
        description="Positions to maintain without changes",
    )
    positions_to_modify: list[Position] = Field(
        default_factory=list,
        description="Positions to modify or close",
    )
    positions_to_open: list[Position] = Field(
        default_factory=list,
        description="Positions to open",
    )

## Make our call

In [11]:
completion = openai_client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    response_format=PositionReasoning,
)

result = completion.choices[0].message.parsed
result.model_dump()

{'positions_to_maintain': [{'market': 'ATOM',
   'direction': 'long',
   'size': 5123.8577,
   'reasoning': ['The ATOM position is small in terms of overall balance and has only a moderate unrealized loss.',
    'There might be potential for ATOM to recover as market sentiment improves.'],
   'leverage': 20},
  {'market': 'BTC',
   'direction': 'short',
   'size': 15224.59344,
   'reasoning': ["The BTC short position has a significant loss, suggesting that the market might have moved against this position already. It's important to evaluate before making any quick decisions, considering broader market trends."],
   'leverage': 20},
  {'market': 'BNB',
   'direction': 'short',
   'size': 11442.21936,
   'reasoning': ['Similar to BTC, the BNB short is under pressure.',
    'Maintaining it for now while monitoring the market closely.'],
   'leverage': 20},
  {'market': 'GMT',
   'direction': 'long',
   'size': 59.26208,
   'reasoning': ["The current size of the position is too small to in

## Submit our trades (Incomplete example for testing)

For positions_to_maintain there is no action required, but it's helpful to log and explain the "reasoning" to users
positions_to_open and positions_to_modify both require submitting orders onchain. 


If you wish to start trading before updates to this notebook are available, you can refer to the [hyperliquid-python-sdk.](https://github.com/hyperliquid-dex/hyperliquid-python-sdk) 

In [12]:
result.positions_to_open

[Position(market='APE', direction='long', size=5375.64177, reasoning=["The APE position is currently at a significant loss. Given the available balance and the potential upside due to Robinhood's increased trading volume, converting this into a long position might capture upside potential.", "Additionally, with overall market conditions looking more favorable, there's an expectation that altcoins like APE could rally."], leverage=5)]

In [13]:
result.positions_to_open[0].market

'APE'

In [14]:
result.positions_to_open[0].direction

'long'

In [15]:
result.positions_to_open[0].size

5375.64177

In [16]:
result.positions_to_open[0].leverage

5

In [17]:
result.positions_to_open[0].reasoning

["The APE position is currently at a significant loss. Given the available balance and the potential upside due to Robinhood's increased trading volume, converting this into a long position might capture upside potential.",
 "Additionally, with overall market conditions looking more favorable, there's an expectation that altcoins like APE could rally."]